# Assessment for Advanced Data Science

## Christian Cabrera, Radzim Sendyka, Carl Henrik Ek and Neil D. Lawrence

### 6th November 2023



Welcome to the course assessment for the Advanced Data Science unit. In this assessment you will build a prediction system for UK house prices.

Your prediction system will be based on data from the UK Price Paid data available [here](https://www.gov.uk/government/statistical-data-sets/price-paid-data-downloads). By combining this data with the UK Office for National Statistics data on the latitude/longitude of postcodes (available [here](https://www.getthedata.com/open-postcode-geo)) you will have a record of house prices and their approximate latitude/longitude. Due to the size of these data you will use a relational database to handle them.  

To make predictions of the house price you will augment your data with information obtained from Open Street Map: an open license source of mapping information. You will use the techniques you have learnt in the course to indentify and incorporate useful features for house price prediction.



Alongside your implementation you will provide a short repository overview describing how you have implemented the different parts of the project and where you have placed those parts in your code repository. You will submit your code alongside a version of this notebook that will allow your examiner to understand and reconstruct the thinking behind your analysis. This notebook is structured to help you in creating that description and allow you to understand how we will allocate the marks. You should make use of the Fynesse framework (<https://github.com/lawrennd/fynesse_template>) for structuring your code.

Remember the notebook you create should *tell a story*, any code that is not critical to that story can safely be placed into the associated analysis library and imported for use (structured as given in the Fynesse template)

The maximum total mark for this assessment is 20. That mark is split into Three Questions below, each worth 5 marks each. Then a final 5 marks will be given for the quality, structure and reusability of the code and analysis you produce giving 20 marks in total. At the end, we would like to know your experience using LLMs in this assignment.

### Important Note:

*Some tasks in this assignment require you to develop skills for searching for multiple solutions and trying different things. This environment recreates industrial data science and software engineering problems. There is no one right answer.*


### Useful Links

You may find some of the following links useful when building your system.

University instuctions on Security and Privacy with AWS.

https://help.uis.cam.ac.uk/service/network-services/hosting-services/AWS/aws-security-privacy

Security Rules in AWS

https://docs.aws.amazon.com/AmazonRDS/latest/UserGuide/USER_VPC.Scenarios.html#USER_VPC.Scenario4

### Installing Your Library

One artefact to be included in your submission is a python library structured according to the "Access, Assess, Address" standard for data science solutions. You will submit this library alongside your code. Use the cell below to perform the necessary installation instructions for your library.

You should base your module on the template repository given by the Fynesse template repository. That should make it `pip` installable as below.



In [ ]:
# Install your library here, for example the fynesse template
# is set up to be pip installable

# TODO: use this when done implementing the functionalities of the library
# %pip install git+https://github.com/lawrennd/fynesse_template.git

Your own library should be installed in the line above, then you can import it as usual (where you can either replace `fynesse` with the name you've given your analysis module or you can leave the name as `fynesse` as you prefer).

In [ ]:
import sys

import pandas as pd

sys.path.append("~/Work/part2/ads/advds/fynesse")
from fynesse.access import *

## Question 1. Accessing a Database of House Prices, Latitudes and Longitudes

The UK price paid data for housing dates back to 1995 and contains millions of transactions. The size of the data makes it unwieldy to manipulate directly in python frameworks such as `pandas`. As a result we will host the data in a *relational database*.

Using the following ideas.

1. A cloud hosted database (such as MariaDB hosted on the AWS RDS service).
2. The SQL language wrapped in appropriately structured python code.
3. Joining of two databases.

You will construct a database containing tables that contain all house prices, latitudes and longitudes from the UK house price data base since 1995.

You will likely find the following resources helpful.

1. Lecture 1, 2 and 3.
2. Lab class 1 and 2.
3. The UK Price Paid data for houses: <https://www.gov.uk/government/statistical-data-sets/price-paid-data-downloads>
4. The UK ONS Data base of postcode latitude and longitudes:  <https://www.getthedata.com/open-postcode-geo>

Below we provide codeboxes and hints to help you develop your answer.

### Important Notes:

*The main knowledge you need to do a first pass through this question will have been taught by the end of Lab Session 2 (31st October 2023). You will likely want to review your answer as part of **refactoring** your code  and analysis pipeline shortly before hand in.*

*We recommend doing Question 1 as early as possible to avoid being blocked from important work given that uploading the data can take long.*

*If you encounter problems with the online notebook (e.g., interrupted connections with the AWS server), you can use a local IDE to work in your machine.*

*5 Marks*

In [96]:
import pymysql
import osmnx as ox
import pandas as pd

config = {
    "default_tags": {
        "amenity": ["school", "restaurant"],
        "leisure": True,
        "healthcare":True,
        "shop": True,
        "public_transport": True
    },
    "default_tags_list": ["school", "restaurant", "leisure", "healthcare", "shop", "public_transport"],
    "default_bounding_box": 0.1,
    "default_category_distance_boundaries": {
        "walking_distance": 0.5,
        "cycling_distance": 5,
        "driving_distance": 10
    }
}

class PropertyPricesDbConnector:
    _conn = None

    def __init__(self, host, port, username, password):
        self._host = host
        self._port = port
        self._username = username
        self._password = password
        self._database = "property_prices"

        try:
            self._create_database()
            self._create_connection_to_database()
        except DatabaseCreationException as database_creation_exception:
            raise Exception(database_creation_exception)
        except DatabaseConnectionException as database_connection_exception:
            raise Exception(database_connection_exception)

    def _create_database(self):
        try:
            conn = pymysql.connect(
                host=self._host,
                port=self._port,
                user=self._username,
                password=self._password
            )
            conn.cursor().execute(f"CREATE DATABASE IF NOT EXISTS {self._database}")
        except Exception as e:
            raise DatabaseCreationException(f"Could not create a new database at the give server! - {e}")

    def _create_connection_to_database(self):
        """
            Create connection to the MariaDB instance on AWS by using the host url, port, credentials and database name.
        """
        try:
            self._conn = pymysql.connect(
                host=self._host,
                port=self._port,
                user=self._username,
                password=self._password,
                database=self._database,
                local_infile=1
            )
        except Exception as e:
            raise DatabaseConnectionException(f"Could not establish connection to the database server! - {e}")

    def get_conn(self):
        return self._conn



class DatabaseCreationException(Exception):
    def __init__(self, message):
        super().__init__(message)


class DatabaseConnectionException(Exception):
    def __init__(self, message):
        super().__init__(message)


def create_property_prices_database(host, port, username, password):
    conn = pymysql.connect(
        host=host,
        port=port,
        user=username,
        password=password
    )
    conn.cursor().execute("CREATE DATABASE IF NOT EXISTS `property_prices`")


def create_property_prices_db_connection(host, port, username, password):
    conn = pymysql.connect(
        host=host,
        port=port,
        user=username,
        password=password,
        database="property_prices",
        local_infile=1
    )
    return conn


def create_and_populate_pp_data_table(conn):
    create_pp_data_table(conn)
    populate_pp_data_table(conn)


def create_pp_data_table(conn):
    conn.cursor().execute("""
        -- Table structure for table `pp_data`
        DROP TABLE IF EXISTS `pp_data`;
    """)
    conn.cursor().execute("""
        CREATE TABLE IF NOT EXISTS `pp_data` (
          `transaction_unique_identifier` tinytext COLLATE utf8_bin NOT NULL,
          `price` int(10) unsigned NOT NULL,
          `date_of_transfer` date NOT NULL,
          `postcode` varchar(8) COLLATE utf8_bin NOT NULL,
          `property_type` varchar(1) COLLATE utf8_bin NOT NULL,
          `new_build_flag` varchar(1) COLLATE utf8_bin NOT NULL,
          `tenure_type` varchar(1) COLLATE utf8_bin NOT NULL,
          `primary_addressable_object_name` tinytext COLLATE utf8_bin NOT NULL,
          `secondary_addressable_object_name` tinytext COLLATE utf8_bin NOT NULL,
          `street` tinytext COLLATE utf8_bin NOT NULL,
          `locality` tinytext COLLATE utf8_bin NOT NULL,
          `town_city` tinytext COLLATE utf8_bin NOT NULL,
          `district` tinytext COLLATE utf8_bin NOT NULL,
          `county` tinytext COLLATE utf8_bin NOT NULL,
          `ppd_category_type` varchar(2) COLLATE utf8_bin NOT NULL,
          `record_status` varchar(2) COLLATE utf8_bin NOT NULL,
          `db_id` bigint(20) unsigned NOT NULL
        ) DEFAULT CHARSET=utf8 COLLATE=utf8_bin AUTO_INCREMENT=1 ;
    """)
    conn.cursor().execute("""
        -- Primary key for table `pp_data` 
        ALTER TABLE `pp_data`
        ADD PRIMARY KEY (`db_id`);
    """)
    conn.cursor().execute("""
        ALTER TABLE `pp_data`
        MODIFY db_id bigint(20) unsigned NOT NULL AUTO_INCREMENT, AUTO_INCREMENT=1;
    """)
    conn.commit()


def populate_pp_data_table(conn):
    conn.cursor().execute("""
        LOAD DATA LOCAL INFILE 'pp-complete.csv' INTO TABLE pp_data
        FIELDS TERMINATED BY ',' OPTIONALLY ENCLOSED by '"'
        LINES STARTING BY '' TERMINATED BY '\n';
    """)
    conn.commit()


def create_column_index_on_pp_data_table(conn, column_name):
    index_column_name = "pp." + column_name

    conn.cursor().execute(f"""
        DROP INDEX IF EXISTS `{index_column_name}` ON `pp_data`
    """)
    conn.cursor().execute(f"""
        CREATE INDEX IF NOT EXISTS `{index_column_name}` USING HASH
        ON `pp_data` ({column_name})
    """)
    conn.commit()

# POSTCODE DATA TABLE

def create_and_populate_postcode_data_table(conn):
    create_postcode_data_table(conn)
    populate_postcode_data_table(conn)


def create_postcode_data_table(conn):
    conn.cursor().execute("""
        -- Table structure for table `postcode_data`
        DROP TABLE IF EXISTS `postcode_data`;
    """)
    conn.cursor().execute("""
        CREATE TABLE IF NOT EXISTS `postcode_data` (
          `postcode` varchar(8) COLLATE utf8_bin NOT NULL,
          `status` enum('live','terminated') NOT NULL,
          `usertype` enum('small', 'large') NOT NULL,
          `easting` int unsigned,
          `northing` int unsigned,
          `positional_quality_indicator` int NOT NULL,
          `country` enum('England', 'Wales', 'Scotland', 'Northern Ireland', 'Channel Islands', 'Isle of Man') NOT NULL,
          `latitude` decimal(11,8) NOT NULL,
          `longitude` decimal(10,8) NOT NULL,
          `postcode_no_space` tinytext COLLATE utf8_bin NOT NULL,
          `postcode_fixed_width_seven` varchar(7) COLLATE utf8_bin NOT NULL,
          `postcode_fixed_width_eight` varchar(8) COLLATE utf8_bin NOT NULL,
          `postcode_area` varchar(2) COLLATE utf8_bin NOT NULL,
          `postcode_district` varchar(4) COLLATE utf8_bin NOT NULL,
          `postcode_sector` varchar(6) COLLATE utf8_bin NOT NULL,
          `outcode` varchar(4) COLLATE utf8_bin NOT NULL,
          `incode` varchar(3)  COLLATE utf8_bin NOT NULL,
          `db_id` bigint(20) unsigned NOT NULL
        ) DEFAULT CHARSET=utf8 COLLATE=utf8_bin;
    """)
    conn.cursor().execute("""
        ALTER TABLE `postcode_data`
        ADD PRIMARY KEY (`db_id`);
    """)
    conn.cursor().execute("""
        ALTER TABLE `postcode_data`
        MODIFY `db_id` bigint(20) unsigned NOT NULL AUTO_INCREMENT,AUTO_INCREMENT=1;
    """)
    conn.commit()


def populate_postcode_data_table(conn):
    conn.cursor().execute("""
        LOAD DATA LOCAL INFILE 'open_postcode_geo.csv' INTO TABLE `postcode_data`
        FIELDS TERMINATED BY ',' OPTIONALLY ENCLOSED by '"'
        LINES STARTING BY '' TERMINATED BY '\n';
    """)
    conn.commit()


def create_column_index_on_postcode_data_table(conn, column_name):
    index_column_name = "pd." + column_name

    conn.cursor().execute(f"""
        DROP INDEX IF EXISTS `{index_column_name}` ON `postcode_data`
    """)
    conn.cursor().execute(f"""
        CREATE INDEX IF NOT EXISTS `{index_column_name}` USING HASH
        ON `postcode_data` ({column_name})
    """)
    conn.commit()



def create_and_populate_prices_coordinates_data_table(conn):
    create_prices_coordinates_data_table(conn)
    populate_prices_coordinates_data_table(conn)


def create_prices_coordinates_data_table(conn):
    conn.cursor().execute("""
        DROP TABLE IF EXISTS `prices_coordinates_data`;
    """)
    conn.cursor().execute("""
        CREATE TABLE IF NOT EXISTS `prices_coordinates_data` (
          `price` int(10) unsigned NOT NULL,
          `date_of_transfer` date NOT NULL,
          `postcode` varchar(8) COLLATE utf8_bin NOT NULL,
          `property_type` varchar(1) COLLATE utf8_bin NOT NULL,
          `new_build_flag` varchar(1) COLLATE utf8_bin NOT NULL,
          `tenure_type` varchar(1) COLLATE utf8_bin NOT NULL,
          `locality` tinytext COLLATE utf8_bin NOT NULL,
          `town_city` tinytext COLLATE utf8_bin NOT NULL,
          `district` tinytext COLLATE utf8_bin NOT NULL,
          `county` tinytext COLLATE utf8_bin NOT NULL,
          `country` enum('England', 'Wales', 'Scotland', 'Northern Ireland', 'Channel Islands', 'Isle of Man') NOT NULL,
          `latitude` decimal(11,8) NOT NULL,
          `longitude` decimal(10,8) NOT NULL,
          `db_id` bigint(20) unsigned NOT NULL
        ) DEFAULT CHARSET=utf8 COLLATE=utf8_bin AUTO_INCREMENT=1 ;
    """)
    conn.cursor().execute("""
        ALTER TABLE `prices_coordinates_data`
        ADD PRIMARY KEY (`db_id`);
    """)
    conn.cursor().execute("""
        ALTER TABLE `prices_coordinates_data`
        MODIFY `db_id` bigint(20) unsigned NOT NULL AUTO_INCREMENT,AUTO_INCREMENT=1;
    """)
    conn.commit()


def populate_prices_coordinates_data_table(conn):
    conn.cursor().execute("""
        INSERT INTO `prices_coordinates_data` (
            `price`,
            `date_of_transfer`,
            `postcode`,
            `property_type`,
            `new_build_flag`,
            `tenure_type`,
            `locality`,
            `town_city`,
            `district`,
            `county`,
            `country`,
            `latitude`,
            `longitude`
        )
        SELECT pp.price, pp.date_of_transfer, pp.postcode, pp.property_type, pp.new_build_flag, pp.tenure_type, pp.locality, pp.town_city, pp.district, pp.county, pd.country, pd.latitude, pd.longitude
        FROM
            (
                SELECT price, date_of_transfer, postcode, property_type, new_build_flag, tenure_type, locality, town_city, district, county
                FROM pp_data 
            ) pp
        INNER JOIN
            (
                SELECT postcode, country, latitude, longitude
                FROM postcode_data
            ) pd
        ON pp.postcode = pd.postcode
    """)
    conn.commit()


def create_column_index_on_prices_coordinates_data_table(conn, column_name):
    index_column_name = "pcd." + column_name
    conn.cursor().execute(f"""
        DROP INDEX IF EXISTS `{index_column_name}` ON `prices_coordinates_data`
    """)
    conn.cursor().execute(f"""
        CREATE INDEX IF NOT EXISTS `{index_column_name}` USING HASH
        ON `prices_coordinates_data` ({column_name})
    """)
    conn.commit()
    

def get_number_of_rows(conn):
    cur = conn.cursor()
    cur.execute("""
       SELECT COUNT(*) AS row_count
       FROM `prices_coordinates_data`
    """)
    return cur.fetchall()


def get_prices_coordinates_for_coords_and_timedelta(conn, bounding_box, min_date, max_date, property_type):
    north, south, west, east = bounding_box
    cur = conn.cursor()
    cur.execute(f"""
        SELECT pp.price, pp.date_of_transfer, pp.postcode, pp.property_type, pp.new_build_flag, pp.tenure_type, pp.locality, pp.town_city, pp.district, pp.county, pd.country, pd.latitude, pd.longitude
        FROM
            (
                SELECT price, date_of_transfer, postcode, property_type, new_build_flag, tenure_type, locality, town_city, district, county 
                FROM pp_data
                WHERE (date_of_transfer BETWEEN '{min_date}' AND '{max_date}') AND property_type = '{property_type}' 
            ) pp
        INNER JOIN
            (
                SELECT postcode, country, latitude, longitude
                FROM postcode_data
                WHERE (latitude BETWEEN {south} AND {north}) AND (longitude BETWEEN {west} AND {east})
            ) pd
        ON pp.postcode = pd.postcode
    """)
    return cur.fetchall()


# OpenStreetMap
def compute_bounding_box_cardinals(latitude, longitude, box_width=config["default_bounding_box"], box_height=config["default_bounding_box"]):
    north = latitude + box_height / 2
    south = latitude - box_height / 2
    west = longitude - box_width / 2
    east = longitude + box_width / 2

    return north, south, west, east


def retrieve_pois_from_bbox_given_tags(bounding_box, tags=config["default_tags"]):
    north, south, west, east = bounding_box
    return ox.features_from_bbox(north, south, east, west, tags)
    

### Task A

Set up the database. You'll need to set up a database on AWS. You were guided in how to do this in the lab sessions. You should be able to use the same database instance you created in the lab, or you can delete that and start with a fresh instance. You'll remember from the lab that the database requires credentials (username, password) to access. It's good practice to store those credentials *outside* the notebook so you don't accidentally share them by e.g. checking code into a repository.
  
Call the database you use for this assessment `property_prices`.
  

In [ ]:
# Write code for requesting and storing credentials (username, password) here.
import yaml
from ipywidgets import interact_manual, Text, Password

@interact_manual(username=Text(description="Username:"), password=Password(description="Password:"))
def store_credentials(username, password):
    with open("credentials.yaml", "w") as file:
        credentials_dict = {'username': username,
                            'password': password}
        yaml.dump(credentials_dict, file)

In [66]:
import yaml
# Write any other setup code you need for setting up database access here.

database_details = {
    "url": "database-ads-sap86.cgrre17yxw11.eu-west-2.rds.amazonaws.com",
    "port": 3306
}

with open("credentials.yaml") as f:
    credentials = yaml.safe_load(f)

username = credentials["username"]
password = credentials["password"]
host = database_details["url"]
port = database_details["port"]

create_property_prices_database(host, port, username, password)
conn = create_property_prices_db_connection(host, port, username, password)

### Task B

Create a database table called `pp_data` containing all the UK Price Paid data from the [gov.uk site](https://www.gov.uk/government/statistical-data-sets/price-paid-data-downloads). You'll need to prepare a new table to receive the data and upload the UK Price Paid data to your database instance. The total data is over 3 gigabytes in size. We suggest that rather than downloading the full data in CSV format, you use the fact that they have split the data into years and into different parts per year. For example, the first part of the data for 2018 is stored at <http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2018-part1.csv>. Each of these files is less than 100MB and can be downloaded very quickly to local disk, then uploaded using


```
LOAD DATA LOCAL INFILE 'filename' INTO TABLE `table_name`
FIELDS TERMINATED BY ',' OPTIONALLY ENCLOSED by '"'
LINES STARTING BY '' TERMINATED BY '\n';
```
*Note* this command should be wrapped and placed in an appropriately structured python module.

Each 'data part' should be downloadable from the `gov.uk` site. It should take around 5 minutes to download the whole dataset. By looping across the years and different parts, you should be able to robustly upload this large data set to your database instance in around 15 minutes. You should get a table with 28'258,161 rows. ***Note: A select count of the table can take more than half an hour.***

You may find the following schema useful in the creation of your table:

```
--
-- Table structure for table `pp_data`
--
DROP TABLE IF EXISTS `pp_data`;
CREATE TABLE IF NOT EXISTS `pp_data` (
  `transaction_unique_identifier` tinytext COLLATE utf8_bin NOT NULL,
  `price` int(10) unsigned NOT NULL,
  `date_of_transfer` date NOT NULL,
  `postcode` varchar(8) COLLATE utf8_bin NOT NULL,
  `property_type` varchar(1) COLLATE utf8_bin NOT NULL,
  `new_build_flag` varchar(1) COLLATE utf8_bin NOT NULL,
  `tenure_type` varchar(1) COLLATE utf8_bin NOT NULL,
  `primary_addressable_object_name` tinytext COLLATE utf8_bin NOT NULL,
  `secondary_addressable_object_name` tinytext COLLATE utf8_bin NOT NULL,
  `street` tinytext COLLATE utf8_bin NOT NULL,
  `locality` tinytext COLLATE utf8_bin NOT NULL,
  `town_city` tinytext COLLATE utf8_bin NOT NULL,
  `district` tinytext COLLATE utf8_bin NOT NULL,
  `county` tinytext COLLATE utf8_bin NOT NULL,
  `ppd_category_type` varchar(2) COLLATE utf8_bin NOT NULL,
  `record_status` varchar(2) COLLATE utf8_bin NOT NULL,
  `db_id` bigint(20) unsigned NOT NULL
) DEFAULT CHARSET=utf8 COLLATE=utf8_bin AUTO_INCREMENT=1 ;
```
This schema is written by Dale Potter and can be found on Github here: <https://github.com/dalepotter/uk_property_price_data/blob/master/create_db.sql>

You may also find it helpful to set up the following primary key to the `pp_data` table. This should be done before uploading your data.

```
--
-- Primary key for table `pp_data`
--
ALTER TABLE `pp_data`
ADD PRIMARY KEY (`db_id`);

ALTER TABLE `pp_data`
MODIFY db_id bigint(20) unsigned NOT NULL AUTO_INCREMENT, AUTO_INCREMENT=1;
```

In the box below, briefly describe what the schema is doing.

The schema is used to create a fresh instance of our required pp_data table.
* It firstly makes sure that any previous instantiation of pp_data is deleted.
* Secondly, it creates a new instance of the table by specifying the columns/fields and their associated types that can be found in the table. It is worth nothing that the column names are carefully chosen such that they match the columns from the downloaded CSV file.

In [4]:
# Write the code you need for creating the table, downloading and uploading the data here. You can use as many code blocks as you need.
create_and_populate_pp_data_table(conn)

### Task C

Create a database table called `postcode_data` containing the ONS Postcode information. <GetTheData.com> has organised data derived from the UK Office for National Statistics into a convenient CSV file. You can find details [here](https://www.getthedata.com/open-postcode-geo).


The data you need can be found at this url: <https://www.getthedata.com/downloads/open_postcode_geo.csv.zip>. It will need to be unzipped before use. Downloading and unzipping the data should not take more than 10 seconds.

You may find the following schema useful for the postcode data (developed by Christian and Neil)

```
USE `property_prices`;
--
-- Table structure for table `postcode_data`
--
DROP TABLE IF EXISTS `postcode_data`;
CREATE TABLE IF NOT EXISTS `postcode_data` (
  `postcode` varchar(8) COLLATE utf8_bin NOT NULL,
  `status` enum('live','terminated') NOT NULL,
  `usertype` enum('small', 'large') NOT NULL,
  `easting` int unsigned,
  `northing` int unsigned,
  `positional_quality_indicator` int NOT NULL,
  `country` enum('England', 'Wales', 'Scotland', 'Northern Ireland', 'Channel Islands', 'Isle of Man') NOT NULL,
  `latitude` decimal(11,8) NOT NULL,
  `longitude` decimal(10,8) NOT NULL,
  `postcode_no_space` tinytext COLLATE utf8_bin NOT NULL,
  `postcode_fixed_width_seven` varchar(7) COLLATE utf8_bin NOT NULL,
  `postcode_fixed_width_eight` varchar(8) COLLATE utf8_bin NOT NULL,
  `postcode_area` varchar(2) COLLATE utf8_bin NOT NULL,
  `postcode_district` varchar(4) COLLATE utf8_bin NOT NULL,
  `postcode_sector` varchar(6) COLLATE utf8_bin NOT NULL,
  `outcode` varchar(4) COLLATE utf8_bin NOT NULL,
  `incode` varchar(3)  COLLATE utf8_bin NOT NULL,
  `db_id` bigint(20) unsigned NOT NULL
) DEFAULT CHARSET=utf8 COLLATE=utf8_bin;
```

And again you'll want to set up a primary key for the new table.

```
ALTER TABLE `postcode_data`
ADD PRIMARY KEY (`db_id`);

ALTER TABLE `postcode_data`
MODIFY `db_id` bigint(20) unsigned NOT NULL AUTO_INCREMENT,AUTO_INCREMENT=1;
```

And you can load the CSV file into the table using this command.

```
LOAD DATA LOCAL INFILE 'open_postcode_geo.csv' INTO TABLE `postcode_data`
FIELDS TERMINATED BY ',' OPTIONALLY ENCLOSED by '"'
LINES STARTING BY '' TERMINATED BY '\n';
```

In [6]:
create_and_populate_postcode_data_table(conn)

### Task D

This table should contain the house price paid and the latitude and longitude of the house. We could create a new data frame that contains all this information. However, the computation of that data frame would take some time because of the size of the two existing tables in the join. Whether this is a good idea or not in a live system will depend on how often these predictions are required. If it's very often, it would be better to store the join as a new table in the database, because the one-off cost for that join is amortised across all the future predictions. If only a few predictions are required (like in our lab class) then doing that join on the fly might be better.

- Option A: Think about which columns from each table will be useful to you in making predictions, then write code to efficiently select this information and create a data frame from the two tables for a set of properties. "Join on the fly". This option looks easier but the disadvantage is the extra latency it adds as joins are performed every time we need to answer data questions. These latencies are usually not accepted in production environments.

- Option B: Alternatively, you can create a database table called `property_prices` to store the join of the tables you created in the previous tasks. The advantage of this approach is that you will get faster responses because the join is performed once. The disadvantage is that populating the new table can take a long time because you would join two big tables. You need to be more creative with this option. Remember that you can divide your dataset by different criteria (e.g., by year) and that loading data from files is much faster than `INSERT INTO` instructions. Populating the table took from 4 to 6 minutes per year in our tests on a Dell Laptop Intel Core i5 with 16GB of RAM and using the Eduroam network at the Computer Lab. Populating the table by year also gives you control over the upload process. You could write your code in a way you can stop and restart the upload process. The new table could have a schema like the one below:

  ```
  USE `property_prices`;
  --
  -- Table structure for table `prices_coordinates_data`
  --
  DROP TABLE IF EXISTS `prices_coordinates_data`;
  CREATE TABLE IF NOT EXISTS `prices_coordinates_data` (
    `price` int(10) unsigned NOT NULL,
    `date_of_transfer` date NOT NULL,
    `postcode` varchar(8) COLLATE utf8_bin NOT NULL,
    `property_type` varchar(1) COLLATE utf8_bin NOT NULL,
    `new_build_flag` varchar(1) COLLATE utf8_bin NOT NULL,
    `tenure_type` varchar(1) COLLATE utf8_bin NOT NULL,
    `locality` tinytext COLLATE utf8_bin NOT NULL,
    `town_city` tinytext COLLATE utf8_bin NOT NULL,
    `district` tinytext COLLATE utf8_bin NOT NULL,
    `county` tinytext COLLATE utf8_bin NOT NULL,
    `country` enum('England', 'Wales', 'Scotland', 'Northern Ireland', 'Channel Islands', 'Isle of Man') NOT NULL,
    `latitude` decimal(11,8) NOT NULL,
    `longitude` decimal(10,8) NOT NULL,
    `db_id` bigint(20) unsigned NOT NULL
  ) DEFAULT CHARSET=utf8 COLLATE=utf8_bin AUTO_INCREMENT=1 ;

  ```

In both cases you will need to perform a join between `pp_data` and `postcode_data` tables. Joining large tables without the right indexes in place will take a long time. You should think and set the right index for an efficient join. Indexing the `pp_data` table should take less than 5 minutes, while it takes less than one minute to index the `postcode_data` table.

Note that there is no preference for either approach in the mark scheme.

You should use the joined data in your prediction model in Question 3. Exploit the nature of the task to use prices for a particular region in a given period. This means we can select the relevant rows from the database according to that region and period.

***After you have populated your database tables and created the functions to access the data you need for Question 3, you will not need to redo any of the previous steps. If at some point you find the AWS database is not responding or taking longer than expected to perform operations, you can have a look at the process list and kill the one are causing problems. If killing the processes does not work, you should reboot the database in the AWS console. Be careful with other database instances if you need to reboot your database. Also, be careful not to delete the database instead of rebooting it. If you delete the database, it is likely you will need to redo all Question 1.***

In [82]:
# Write the code you used to join the tables, or the code you used to join on the fly.

# Firstly, I will create indexes for `postcode` on both tables, as we will be using them to join the tables. Also on date_of_transfer for pp_data.
create_column_index_on_pp_data_table(conn, "postcode")
create_column_index_on_pp_data_table(conn, "date_of_transfer")
create_column_index_on_postcode_data_table(conn, "postcode")

# Secondly, let's create the `price_coordinates_data table in our database
create_prices_coordinates_data_table(conn)

# TODO: Continue with actually populating method

## Question 2. Accessing OpenStreetMap and Assessing the Available Features

In question 3 you will be given the task of constructing a prediction system for property price levels at a given location. We expect that knowledge of the local region around the property should be helpful in making those price predictions. To evaluate this we will now look at [OpenStreetMap](https://www.openstreetmap.org) as a data source.

The tasks below will guide you in accessing and assessing the OpenStreetMap data. The code you write will eventually be assimilated in your python module, but documentation of what you've included and why should remain in the notebook below.

Accessing OpenStreetMap through its API can be done using the python library `osmx`. Using what you have learned about the `osmx` interface in the lectures, write general code for downloading points of interest and other relevant information that you believe may be useful for predicting house prices. Remembering the perspectives we've taken on *data science as debugging*, the remarks we've made when discussing *the data crisis* of the importance of reusability in data analysis, and the techniques we've explored in the labsessions for visualising features and exploring their correlation use the notebook to document your assessment of the OpenStreetMap data as a potential source of data.

The knowledge you need to do a first pass through this question will have been taught by end of lab session three (16th November 2021). You will likely want to review your answer as part of *refactoring* your code and analysis pipeline shortly before hand in.

You should write reusable code that allows you to explore the characteristics of different points of interest. Looking ahead to question 3 you'll want to incorporate these points of interest in your prediction code.

*5 marks*


In [114]:
# assess
from haversine import haversine, Unit

def compute_tags_count_per_distance_category(pois_df, latitude, longitude, tags_list=config["default_tags_list"], category_distance_boundaries=config["default_category_distance_boundaries"]):
    
    tag_count_per_distance_category = {}
    
    for tag in tags_list:
        try:
            pois_by_tag = pois_df[pois_df[tag].notnull()]
        except Exception:
            for category_id, category_distance in category_distance_boundaries.items():
                 tag_count_per_distance_category[str(tag) + "-" + category_id] = 0
            continue
        
        previous_matched_len = 0
        
        for category_id, category_distance in category_distance_boundaries.items():
            matched_pois = pois_by_tag[pois_by_tag["geometry"].apply(lambda geom: haversine((geom.centroid.x, geom.centroid.y), (longitude, latitude), unit=Unit.KILOMETERS) <= category_distance)]
            
            tag_count_per_distance_category[str(tag) + "-" + category_id] = len(matched_pois) - previous_matched_len
            previous_matched_len = len(matched_pois)
            
    return tag_count_per_distance_category
    
    
def display_corr_between_features_and_price(conn, bounding_box, min_date, max_date, house_type):
    house_rows = get_prices_coordinates_for_coords_and_timedelta(conn, bounding_box, min_date, max_date, house_type)
    
    houses_df = pd.DataFrame(
        data=house_rows,
        columns=["price", "date_of_transfer", "postcode", "property_type", "new_build_flag", "tenure_type", "locality", "town_city", "district", "county", "country", "latitude","longitude"]
    )
    return houses_df


As mentioned, integrating POIs will help us understand better the price of a house in a given region. Therefore, after investigating what points of interests are important around your house, I have decided to investigate pois having the following tags:
* amenity : school
* amenity : restaurant
* leisure
* healthcare
* shop
* public_transport

What will I do with these POIs? I will break them down into three categories each, distance wise from our house location. 
* First category counts how many of each are within walking distance, which after researching various studies, proves to be around 0.5 kilometers. 
* Second category counts how many of each individual POIs' tags are within 5 kilometers, which can be considered cycling distance.
* Third category, driving distance within 10 kilometers.

I am going to compute the distance from our point to any POI using haversine distance. Ideally, we would like to compute this via actual road distance, but this would be quite difficult and would possibly not yield very different results, as the query usually spans a small area of a city.

Obviously, we will consider these three categories disjointed, i.e. a location that is within 0.5 kilometers will only be accounted for the first category and not for the remaining two.

In [104]:
# Use this cell and cells below for summarising your analysis and documenting your decision making.
cambridge_longitude = 0.121773
cambridge_latitude = 52.205296
bounding_box = compute_bounding_box_cardinals(cambridge_latitude, cambridge_longitude)

cambridge_pois = retrieve_pois_from_bbox_given_tags(
    compute_bounding_box_cardinals(cambridge_latitude, cambridge_longitude)
)

cambridge_pois.head()

name public_transport  \
element_type osmid                                                         
node         15625999                   Cambridge North    stop_position   
             18629026                The Galfrid School              NaN   
             18629028  The Fields Early Learning Centre              NaN   
             20458873                     Well Pharmacy              NaN   
             20823646                    Hobbs Pavilion              NaN   

                      railway  ref train                  geometry note  \
element_type osmid                                                        
node         15625999    stop    1   yes  POINT (0.15823 52.22355)  NaN   
             18629026     NaN  NaN   NaN  POINT (0.16180 52.20880)  NaN   
             18629028     NaN  NaN   NaN  POINT (0.16207 52.20802)  NaN   
             20458873     NaN  NaN   NaN  POINT (0.14560 52.19752)  NaN   
             20823646     NaN  NaN   NaN  POINT (0.12613 52.20262)  NaN   

                       addr:city addr:postcode   addr:street  ... blind  \
element_type osmid                                            ...         
node         15625999        NaN           NaN           NaN  ...   NaN   
             18629026  Cambridge       CB5 8ND  Galfrid Road  ...   NaN   
             18629028  Cambridge       CB5 8ND  Galfrid Road  ...   NaN   
             20458873  Cambridge       CB1 3AN     Mill Road  ...   NaN   
             20823646  Cambridge       CB1 1JH  Park Terrace  ...   NaN   

                      playground attraction service:bicycle:second_hand  \
element_type osmid                                                        
node         15625999        NaN        NaN                         NaN   
             18629026        NaN        NaN                         NaN   
             18629028        NaN        NaN                         NaN   
             20458873        NaN        NaN                         NaN   
             20823646        NaN        NaN                         NaN   

                      bathing_water informal supervised theatre:type ways type  
element_type osmid                                                              
node         15625999           NaN      NaN        NaN          NaN  NaN  NaN  
             18629026           NaN      NaN        NaN          NaN  NaN  NaN  
             18629028           NaN      NaN        NaN          NaN  NaN  NaN  
             20458873           NaN      NaN        NaN          NaN  NaN  NaN  
             20823646           NaN      NaN        NaN          NaN  NaN  NaN  

[5 rows x 363 columns]

In [64]:
tags_category_count = compute_tags_count_per_distance_category(cambridge_pois, cambridge_latitude, cambridge_longitude)

In [84]:
houses_df = display_corr_between_features_and_price(conn, bounding_box, '2021-01-01', '2021-12-31', 'T')

In [137]:
random_100_houses = houses_df.sample(n=100)

In [138]:
def get_distances_features_from_a_house(random_house):
    print(float(random_house.latitude), float(random_house.longitude))
    bounding_box = compute_bounding_box_cardinals(float(random_house.latitude), float(random_house.longitude))
    house_pois = retrieve_pois_from_bbox_given_tags(bounding_box)
    # print(house_pois.columns.values)
    
    return compute_tags_count_per_distance_category(
        house_pois,
        random_house.latitude,
        random_house.longitude
    )

In [139]:
new_columns_dict = random_100_houses.apply(lambda random_house: get_distances_features_from_a_house(random_house), axis=1)

52.207491 0.141409
52.170691 0.114253
52.218317 0.138943
52.197819 0.151858
52.167304 0.120614
52.221593 0.133111
52.213691 0.116251
52.214644 0.11372
52.240446 0.158974
52.232879 0.130739
52.217731 0.106532
52.194003 0.107443
52.18793 0.139123
52.221393 0.154698
52.199696 0.140623
52.168459 0.123081
52.210229 0.144906
52.205025 0.141524
52.224828 0.126473
52.232802 0.11754
52.227004 0.118948
52.214084 0.146732
52.216165 0.16863
52.201377 0.136196
52.214989 0.159234
52.201505 0.149682
52.205751 0.128488
52.167022 0.102409
52.213015 0.115326
52.197288 0.106765
52.197819 0.151858
52.204855 0.140241
52.219844 0.107042
52.196271 0.149195
52.2138 0.127689
52.21534 0.156526
52.167324 0.122984
52.212171 0.145645
52.228062 0.135575
52.197 0.15138
52.221332 0.088341
52.179347 0.12246
52.23844 0.084708
52.199804 0.143584
52.200214 0.14599
52.188683 0.148127
52.217114 0.11814
52.159936 0.124985
52.230152 0.139613
52.209599 0.141465
52.165204 0.10984
52.203627 0.149184
52.199804 0.143584
52.198213

In [140]:
new_columns_df = pd.DataFrame(new_columns_dict.tolist())

random_100_houses.reset_index(drop=True, inplace=True)
new_columns_df.reset_index(drop=True, inplace=True)

extended_random_100_houses = pd.concat([random_100_houses, new_columns_df], axis=1)

In [141]:
filtered_extended_random_houses = extended_random_100_houses[[
    'price',
    'school-walking_distance', 'school-cycling_distance', 'school-driving_distance', 
    'restaurant-walking_distance', 'restaurant-cycling_distance', 'restaurant-driving_distance',
    'leisure-walking_distance', 'leisure-cycling_distance', 'leisure-driving_distance',
    'healthcare-walking_distance', 'healthcare-cycling_distance', 'healthcare-driving_distance',
    'shop-walking_distance', 'shop-cycling_distance', 'shop-driving_distance',
    'public_transport-walking_distance', 'public_transport-cycling_distance', 'public_transport-driving_distance'
]]

In [143]:
filtered_extended_random_houses.fillna(0, inplace=True)
filtered_extended_random_houses.corr()

/var/folders/zw/k_yg06y900n5xjbfnsrj3mlw0000gn/T/ipykernel_35806/374109193.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_extended_random_houses.fillna(0, inplace=True)


,price,school-walking_distance,school-cycling_distance,school-driving_distance,restaurant-walking_distance,restaurant-cycling_distance,restaurant-driving_distance,leisure-walking_distance,leisure-cycling_distance,leisure-driving_distance,healthcare-walking_distance,healthcare-cycling_distance,healthcare-driving_distance,shop-walking_distance,shop-cycling_distance,shop-driving_distance,public_transport-walking_distance,public_transport-cycling_distance,public_transport-driving_distance
price,1.000000,NaN,-0.122061,0.064665,-0.081403,0.074736,0.131599,0.255819,0.246288,-0.140864,0.256657,0.011503,-0.084300,0.176157,0.016327,-0.060197,0.225897,0.134529,-0.003166
school-walking_distance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
school-cycling_distance,-0.122061,NaN,1.000000,-0.578775,-0.193244,0.161024,0.195638,0.003715,0.217106,-0.108804,0.508978,0.715500,-0.627763,0.559285,0.593483,-0.590211,-0.132227,0.605116,-0.397547
school-driving_distance,0.064665,NaN,-0.578775,1.000000,0.317137,-0.291267,-0.015823,0.011865,0.155336,-0.163708,-0.269153,-0.089018,0.015019,-0.308246,-0.014978,0.021740,0.230659,-0.089606,-0.104239
restaurant-walking_distance,-0.081403,NaN,-0.193244,0.317137,1.000000,-0.134528,-0.073654,0.112763,-0.028777,0.106191,-0.126839,-0.126867,0.099346,-0.124540,-0.111980,0.100351,0.073312,-0.116477,0.082093
restaurant-cycling_distance,0.074736,NaN,0.161024,-0.291267,-0.134528,1.000000,-0.485520,0.209610,0.120894,0.291258,0.168586,-0.068043,0.051090,0.209605,-0.088779,0.094113,-0.162510,0.162021,0.152678
restaurant-driving_distance,0.131599,NaN,0.195638,-0.015823,-0.073654,-0.485520,1.000000,-0.300691,0.091350,-0.208325,0.186022,0.231065,-0.194314,0.230088,0.165941,-0.193028,0.123741,0.131297,-0.081095
leisure-walking_distance,0.255819,NaN,0.003715,0.011865,0.112763,0.209610,-0.300691,1.000000,0.182382,-0.299148,0.177583,0.044057,-0.075928,0.122003,0.091302,-0.140273,0.095641,0.149188,-0.214154
leisure-cycling_distance,0.246288,NaN,0.217106,0.155336,-0.028777,0.120894,0.091350,0.182382,1.000000,-0.681711,0.517467,0.644483,-0.689547,0.501499,0.732175,-0.714636,0.326116,0.838515,-0.728388
leisure-driving_distance,-0.140864,NaN,-0.108804,-0.163708,0.106191,0.291258,-0.208325,-0.299148,-0.681711,1.000000,-0.352302,-0.523459,0.544860,-0.326288,-0.600556,0.621591,-0.307148,-0.553282,0.723730


We observe that almost all of these metrics (except for restaurant-walking_distance) are correlated with the house price. I will keep all of these metrics in the address part, as the could be relevant on a larger scale map or on different box boundaries of different cities/areas.

## Question 3. Addressing a Property Price Prediction Question

For your final tick, we will be asking you to make house price predictions for a given location, date and property type in the UK. You will provide a function that takes input a latitude and longitude as well as the `property_type` (either type" of property (either `F` - flat, `S` - semidetached, `D` - detached, `T` - terraced or `O` other). Create this function in the `address.py` file, for example in the form,

```
def predict_price(latitude, longitude, date, property_type):
    """Price prediction for UK housing."""
    pass
```

We suggest that you use the following approach when building your prediction.

1. Select a bounding box around the housing location in latitude and longitude.
2. Select a data range around the prediction date.
3. Use the data ecosystem you have build above to build a training set from the relevant time period and location in the UK. Include appropriate features from OSM to improve the prediction.
4. Train a linear model on the data set you have created.
5. Validate the quality of the model.
6. Provide a prediction of the price from the model, warning appropriately if your validation indicates the quality of the model is poor.

Please note that the quality of predictions is not the main focus of the assignment - we expect to see models that output reasonable predictions and have positive R^2's, but you should not spend too much time on increasing the model's accuracy.

The knowledge you need to do a first pass through this question will have been taught by end of lab session four (7th November 2023). You will likely want to review your answer as part of *refactoring* your code shortly before hand in.



## Large Language Models

If you used LLMs to generate or fix code in this assignment (recommended), briefly summarise the process and prompts you used. What do you think of the integration of LLMs in the data science pipeline?

```GIVE YOUR WRITTEN ANSWER HERE```

### FAQs

- Why is my connection to the AWS server intermittent?

If you are using Google Colab, online notebooks may throttle your code or time you out. Local Python code is recommended for tasks for large data management in the database.

- Why do SQL queries take a long time?

Some queries legitimately take a long time, even when done right. We suggest indexing your tables to speed up queries over your database. You can index tables by different columns depending on the queries you want to perform. For example, indexing the tables by `postcode` could speed up the join in Task D. More information regarding indexing in MariaDB is available [here](https://mariadb.com/kb/en/getting-started-with-indexes/).

You may also want to consider creating single or multi-column indices on coordinates, or any other properties you use to select data, if that step is taking a long time.

If your new queries seem stuck, try running `SHOW FULL PROCESSLIST`, and `KILL` any stuck processes.

- Why are table populating processes taking so long?

Again populating the database can take long. However, be careful if you are indexing the tables. You should populate data before indexing. Insert operations are impacted by indexes as they are updated with each new row inserted into the table.

- Some other questions are answered in [this reddit forum](https://www.reddit.com/r/CST_ADS/) or [this doc](https://docs.google.com/document/d/1GfDROyUW8HVs2eyxmJzKrYGRdVyUiVXzPcDfwOO8wX0/edit?usp=sharing). Feel free to also ask about anything that comes up.